In [34]:
from scipy.signal import hilbert
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

### FUNCTIONS:

In [ ]:
def shift_relative_time(event, plot=False):
    """
    Shift all channels in an event by the value of the first trigger signal time.

    NOTE: Might be too expensive, as you have to search throughout all of the time arrays to find the minimum time value.
    Perhaps its better to not search? Looking at the generated voltage vs. time arrays, all channel seem to have the same starting time?

    Parameters:
    event (np.array): Event containing channels with voltage vs. time arrays.
    plot (bool): If True, generates before and after subplots (for debugging purposes).
    """
    
    # Plot before the shift, if requested
    if plot:
        _, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].plot(event[0][0], event[0][1])
        ax[0].set_title('Before Shift')
        ax[0].set_xlabel('Time')
        ax[0].set_ylabel('Voltage')

    # Find the minimum time across all channels
    min_time = min(np.min(channel[0]) for channel in event)

    # Subtract min_time from the time values of each channel
    for channel in event:
        channel[0] -= min_time

    # Plot after the shift, if requested
    if plot:
        ax[1].plot(event[0][0], event[0][1])
        ax[1].set_title('After Shift')
        ax[1].set_xlabel('Time (ns)')
        ax[1].set_ylabel('Voltage (V)')
        plt.tight_layout()
        plt.savefig('Before_and_after_shift.png')

In [ ]:
def bin_hilbert(channel, nbins, method='max', plot=False):
    """
    Bin Hilbert envelope vs. time data. The bin values are calculated according to different methods.

    A big issue is that if the different channels effectively HAVE different time domains, then the different channels will contain
    different bin sizes. We could make all bins the same size, but then graphs will have more or less bins, or we could keep it like this.
    Parameters:
    channel (np.array): Channel array containing two numpy arrays of time (pos [0]) and voltage (pos [1]).
    method (str): Modifies bin values according to the method. 
                  'max' - Bins Hilbert envelope according to the local maximum of their respective time bins.
                  'avg' - Bins Hilbert envelope according to the average of their respective time bins.
    plot (bool): If True, plots the original Hilbert envelope and the binned data.

    Returns:
    np.array: Binned time array
    np.array: Binned Hilbert envelope array
    float: Time of a bin.
    """
    
    time_arr = channel[0]
    voltage_arr = channel[1]

    # Obtain Hilbert envelope
    hilb_arr = np.abs(hilbert(voltage_arr))
    
    bin_width = len(voltage_arr) // nbins
    bin_dt = time_arr[bin_width]-time_arr[0]
    binned_time = time_arr[:nbins * bin_width:bin_width]

    if method == 'max':
        binned_hilb = np.array([np.max(hilb_arr[i:i+bin_width]) for i in range(0, nbins * bin_width, bin_width)])
    elif method == 'avg':
        binned_hilb = np.array([np.mean(hilb_arr[i:i+bin_width]) for i in range(0, nbins * bin_width, bin_width)])
    else:
        raise ValueError(f"Invalid method '{method}'. Use 'max' or 'avg'.")

    if plot:
        plt.figure(figsize=(12, 6))
        plt.plot(time_arr, hilb_arr, label='Original Hilbert Envelope', alpha=0.7)
        plt.plot(binned_time, binned_hilb, 'o-', label=f'Binned Hilbert Envelope ({method})', color='red')
        plt.xlabel(f'Time (ns), one bin = {bin_dt} ns')
        plt.ylabel('Hilbert Envelope (V)')
        plt.title('Original vs Binned Hilbert Envelope')
        plt.legend()
        plt.grid(True)
        plt.savefig('Original vs Binned Hilbert Envelope.png')

    return binned_time, binned_hilb, bin_dt

In [ ]:
def calculate_noise(voltage_arr, plot=False):
    """
    Calculate the RMS noise of a voltage array and optionally plot the voltage data.

    The noise is calculated by:
    1. Identifying the peak signal (maximum value) in the array.
    2. Defining a range around the peak, covering 25% of the array's length to the left and right of the peak.
    3. Extracting the noise subarrays from the portions of the array outside this range.
    4. Calculating the RMS noise from the concatenated noise subarrays.

    Parameters:
    voltage_arr (np.array): Array of voltage values.
    plot (bool): If True, plots the voltage data with noise range markers (For debugging purposes).

    Returns:
    float: The RMS noise value.
    """
    
    peak_idx = np.argmax(voltage_arr)
    quarter_len = len(voltage_arr) // 4

    # Define the range around the peak signal
    left_idx = max(0, peak_idx - quarter_len)
    right_idx = min(len(voltage_arr), peak_idx + quarter_len)

    # Extract noise subarrays and concatenate them
    noise_subarray = np.concatenate((voltage_arr[:left_idx], voltage_arr[right_idx:]))

    # Calculate RMS noise
    rms_noise = np.sqrt(np.mean(noise_subarray ** 2))

    # For testing purposes:
    if plot:
        # Plotting the original voltage data
        plt.figure(figsize=(10, 6))
        plt.plot(voltage_arr, label='Voltage Data', color='blue')
        
        # Mark the start and end indices of the noise range
        plt.axvline(x=left_idx, color='red', linestyle='--', label='Left Noise Index')
        plt.axvline(x=right_idx, color='green', linestyle='--', label='Right Noise Index')
        
        # Highlighting the noise areas on the plot
        plt.fill_between(range(left_idx), voltage_arr[:left_idx], color='red', alpha=0.3)
        plt.fill_between(range(right_idx, len(voltage_arr)), voltage_arr[right_idx:], color='green', alpha=0.3)
        
        plt.xlabel('Index')
        plt.ylabel('Voltage (V)')
        plt.title('Voltage Data with Noise Range')
        plt.legend()
        plt.grid(True)
        plt.savefig('Voltage Data with Noise Range.png')

    return rms_noise

In [ ]:
def plot_image(event, nbins):
    """
    Generates a heat map with the binned Hilbert envelope data of the signal. 
    The heat map has channel number on the vertical axis, time on the horizontal axis, 
    and voltage on the color axis.
    
    Parameters:
    event (np.array): Event containing channels with voltage vs. time arrays.
    nbins (int): Number of bins for the Hilbert envelope.
    """
    shift_relative_time(event)
    voltage_matrix = []
    time_bins = []
    
    for channel in event:
        _, voltage_bin, bin_dt = bin_hilbert(channel, nbins, plot=False)
        voltage_matrix.append(voltage_bin)
        time_bins.append(bin_dt)

    # Convert the list of voltage bins to an array
    voltage_matrix = np.array(voltage_matrix)

    # Plot the heatmap
    plt.figure(figsize=(10, 6))
    plt.imshow(voltage_matrix, aspect='auto', cmap='viridis', interpolation='nearest')
    # Set x and y axis labels
    plt.xlabel(f'dt: ({time_bins[0]} ns)')
    plt.ylabel('Channel')
    plt.colorbar(label='Hilbert Voltage (V)')

    # Set y-ticks to show channel numbers
    plt.yticks(ticks=np.arange(0, len(event)), labels=np.arange(0, len(event)))

    plt.savefig('heatmap_test.png')

In [80]:
import os
import pickle

event_data_to_append = {}

def save_events(file_path='data/event_data.pkl',events_in=None):
    """
    Save events in a dictionary. If a dictionary already exists, it will append the events to the end of the dictionary.
    Otherwise, it will create the pickled dictionary in the file_path.

    Parameters:
    file_path (String) location of dictionary, or location where to create it.
    events_in (dict) python dictionary from which to append the events or to save.
    """
    # Check if the file exists
    if os.path.exists(file_path):
        # Load and return the dictionary
        with open(file_path, 'rb') as file:
            event_dict = pickle.load(file)

        start_i = max(event_dict.keys())

        # Function to shift keys by a given number
        def shift_keys(d, shift_amount):
            return {k + shift_amount: v for k, v in d.items()}
        
        shifted_events_in = shift_keys(events_in,start_i+1)
        new_event_dict = {**event_dict, **shifted_events_in}

        with open(file_path, 'wb') as file:
            pickle.dump(new_event_dict, file)
    else:
        # Save the event dictionary to file 
        with open(file_path, 'wb') as file:
            pickle.dump(events_in, file)

### TESTING:

In [81]:
with open('data/saved_dictionary.pkl', 'rb') as file:
    trace1 = pickle.load(file)

with open('data/event_dict.pkl','rb') as file:
    trace2 = pickle.load(file)

print(trace2)

{0: 100, 1: 200, 2: 300, 3: 400}
{0: 100, 1: 200, 2: 300, 3: 400, 4: array([[[4.56815974e+03, 4.56915974e+03, 4.57015974e+03, ...,
         5.23715974e+03, 5.23815974e+03, 5.23915974e+03],
        [8.50662718e-07, 5.54271176e-06, 1.47038393e-05, ...,
         1.82250960e-05, 1.03770512e-05, 2.80333223e-06]],

       [[4.56815974e+03, 4.56915974e+03, 4.57015974e+03, ...,
         5.23715974e+03, 5.23815974e+03, 5.23915974e+03],
        [4.64998987e-06, 1.06219203e-05, 6.51222314e-06, ...,
         2.07249902e-05, 8.70961913e-06, 1.83651711e-06]],

       [[4.56815974e+03, 4.56915974e+03, 4.57015974e+03, ...,
         5.23715974e+03, 5.23815974e+03, 5.23915974e+03],
        [9.43199333e-07, 6.92773880e-06, 2.15477143e-06, ...,
         1.20616352e-05, 1.21008256e-05, 8.63860394e-06]],

       [[4.56815974e+03, 4.56915974e+03, 4.57015974e+03, ...,
         5.23715974e+03, 5.23815974e+03, 5.23915974e+03],
        [1.37222616e-05, 1.47496323e-06, 1.19290619e-05, ...,
         8.65415509e-06

In [79]:
# Function testing block

with open('data/saved_dictionary.pkl', 'rb') as file:
    trace1 = pickle.load(file) # WARNING: The pickle module is not secure. Only unpickle data you trust.

# def shift_keys(d, shift_amount):
#     return {k + shift_amount: v for k, v in d.items()}

# print(tracey)

# tracey = shift_keys(tracey,-1)

# print(tracey)

# with open('data/saved_dictionary.pkl', 'wb') as file:
#     pickle.dump(tracey, file) # WARNING: The pickle module is not secure. Only unpickle data you trust.

save_events('data/event_dict.pkl',trace1)

with open('data/event_dict.pkl','rb') as file:
    trace2_changed = pickle.load(file)



print(trace2_changed)

{0: 100, 1: 200, 2: 300, 3: 400}


In [39]:
# Most pressing issue:
# Identify whether or not events and channels can have different time domains. If not, then we are good, otherwise must find a work around as binning will be different for different channels.
# Identify optimal function structure. Perhaps, plot image should only plot images, not generate arrays?
# Begin testing!!
arr = np.array([])
np.append(arr, np.array([1,2,3]))
np.append(arr, np.array([4,5,6]))

print(arr)

[]
